In [2]:
import subprocess
import time
import random

## Evaluating One Candidate

In [ ]:
def compile_to_binary(input_bc, output_bin,
                       clang_bin="clang",
                       target="riscv64",
                       march="rv64imaf",
                       mabi="lp64",
                       static_link=True,
                       includes=["-I./support/"],
                    #    extra_sources=["./support/beebsc.c", "./support/main.c"],
                       extra_sources=None,
                       extra_flags=None):
    
    cmd = [
        clang_bin,
        "-target", target,
        "--sysroot=/opt/riscv_ima/riscv64-unknown-elf",
        input_bc,
        "-o", output_bin,
        f"-march={march}",
        f"-mabi={mabi}"
    ]

    if static_link:
        cmd.append("-static")

    if includes:
        cmd += includes

    if extra_sources:
        cmd += extra_sources

    if extra_flags:
        cmd += extra_flags

    print(f"Compiling to RISC-V binary: {output_bin}")
    print("Running:", " ".join(cmd))
    # print(" ".join(cmd))
    subprocess.run(cmd, check=True)


def apply_passes(input_bc, output_bc, pass_order):
    
    passes_str = ",".join(pass_order)
    cmd = ["opt", f"-passes={passes_str}", input_bc, "-o", output_bc]
    print(f"Applying passes: {passes_str}")
    print("Running:", " ".join(cmd))
    subprocess.run(cmd, check=True)


import subprocess

def compile_to_bitcode(
    sources,
    output_bc,
    clang_bin="clang",
    target="riscv64",
    includes=["-I./support/"],
    extra_flags=None
):
    cmd = [
        clang_bin,
        "-target", target,
        "-march=rv64ima",
        "-mabi=lp64",
        "--sysroot=/opt/riscv_ima/riscv64-unknown-elf",
        "-emit-llvm",
        "-c",
        "-O0"  # Emit unoptimized bitcode
    ]

    if includes:
        cmd += includes

    if extra_flags:
        cmd += extra_flags

    cmd += sources
    cmd += ["-o", output_bc]

    print(f"Compiling to LLVM bitcode: {output_bc}")
    print("Running:", " ".join(cmd))
    subprocess.run(cmd, check=True)


In [47]:
# def apply_passes(input_bc, output_bc, pass_order):
#     passes_str = ",".join(pass_order)
#     cmd = ["opt", f"-passes={passes_str}", input_bc, "-o", output_bc]
#     subprocess.run(cmd, check=True)

# def compile_to_binary(output_bc, output_bin):
#     cmd = ["clang", output_bc, "-o", output_bin]
#     subprocess.run(cmd, check=True)

def measure_runtime(binary, count=10):
    full_time = 0
    for i in range(count):
        start = time.time()
        subprocess.run([f"qemu-riscv64 ./{binary}"], check=True)
        end = time.time()
        full_time += (end - start)
    return full_time/count

def measure_size(binary):
    result = subprocess.run(["size", binary], capture_output=True, text=True)
    lines = result.stdout.strip().split("\n")
    if len(lines) < 2:
        return None
    size_fields = lines[1].split()
    total_size = sum(int(x) for x in size_fields[:4])
    return total_size

def evaluate_candidate(pass_order, input_bc, workdir, mode="runtime"):
    candidate_bc = f"{workdir}/candidate.bc"
    candidate_bin = f"{workdir}/candidate_bin"
    try:
        apply_passes(input_bc, candidate_bc, pass_order)
        compile_to_binary(candidate_bc, candidate_bin)
        if mode == "runtime":
            fitness = measure_runtime(candidate_bin)
        elif mode == "size":
            fitness = measure_size(candidate_bin)
        else:
            raise ValueError("Invalid mode")
        return fitness
    except subprocess.CalledProcessError:
        return float("inf")  # Penalize failure

In [48]:
compile_to_binary("./scratch/test.bc", "./scratch/test_bin")

Compiling to RISC-V binary: ./scratch/test_bin
Running: clang -target riscv64 --sysroot=/opt/riscv_ima/riscv64-unknown-elf ./scratch/test.bc -o ./scratch/test_bin -march=rv64imaf -mabi=lp64 -static -I./support/ ./support/beebsc.c ./support/main.c
clang -target riscv64 --sysroot=/opt/riscv_ima/riscv64-unknown-elf ./scratch/test.bc -o ./scratch/test_bin -march=rv64imaf -mabi=lp64 -static -I./support/ ./support/beebsc.c ./support/main.c


./support/main.c:54:5: warning: implicit declaration of function 'initialise_benchmark' is invalid in C99 [-Wimplicit-function-declaration]
    initialise_benchmark();
    ^
./support/main.c:57:5: warning: implicit declaration of function 'warm_caches' is invalid in C99 [-Wimplicit-function-declaration]
    warm_caches(1);
    ^
./support/main.c:60:14: warning: implicit declaration of function 'benchmark' is invalid in C99 [-Wimplicit-function-declaration]
    errors = benchmark();
             ^
./support/main.c:63:9: warning: implicit declaration of function 'verify_benchmark' is invalid in C99 [-Wimplicit-function-declaration]
    if (verify_benchmark(errors)) {
        ^
./support/main.c:64:9: warning: implicitly declaring library function 'printf' with type 'int (const char *, ...)' [-Wimplicit-function-declaration]
        printf("Benchmark completed successfully with no errors.\n");
        ^
./support/main.c:64:9: note: include the header <stdio.h> or explicitly provide a decla

CalledProcessError: Command '['clang', '-target', 'riscv64', '--sysroot=/opt/riscv_ima/riscv64-unknown-elf', './scratch/test.bc', '-o', './scratch/test_bin', '-march=rv64imaf', '-mabi=lp64', '-static', '-I./support/', './support/beebsc.c', './support/main.c']' returned non-zero exit status 1.

## Genetic Algorithm Operators

In [36]:

def random_candidate(pass_pool, max_len=5):
    length = random.randint(1, max_len)
    return random.sample(pass_pool, length)



def crossover(parent1, parent2, max_len=5):
    # Pick cut points
    p1_cut = random.randint(1, len(parent1))
    p2_cut = random.randint(1, len(parent2))

    # Combine slices
    child = parent1[:p1_cut] + parent2[p2_cut:]

    # Remove duplicates while preserving order
    seen = set()
    deduped_child = []
    for p in child:
        if p not in seen:
            deduped_child.append(p)
            seen.add(p)

    # Clip to max length
    return deduped_child[:max_len]


def mutate(candidate, pass_pool, mutation_rate=0.1, max_len=5):
    # Start with a copy
    mutated = candidate[:]
    
    # Replace existing passes
    for i in range(len(mutated)):
        if random.random() < mutation_rate:
            # Choose a new pass not in the candidate
            available = list(set(pass_pool) - set(mutated))
            if available:
                mutated[i] = random.choice(available)

    # Random insertion
    if len(mutated) < max_len and random.random() < mutation_rate:
        available = list(set(pass_pool) - set(mutated))
        if available:
            insert_pos = random.randint(0, len(mutated))
            mutated.insert(insert_pos, random.choice(available))

    # Random deletion
    if len(mutated) > 1 and random.random() < mutation_rate:
        del_pos = random.randint(0, len(mutated) - 1)
        mutated.pop(del_pos)

    return mutated



## Running the GA loop

In [37]:
def run_ga(input_bc, pass_pool, generations=10, pop_size=20, seq_length=5):
    population = [random_candidate(pass_pool, seq_length) for _ in range(pop_size)]

    for generation in range(generations):
        print(f"Generation {generation}")
        fitnesses = []
        for candidate in population:
            # fitness = evaluate_candidate(candidate, input_bc, "/tmp/workdir")
            fitness = evaluate_candidate(candidate, input_bc, "./scratch")
            fitnesses.append((fitness, candidate))
            print(f"Candidate {candidate} => Fitness {fitness}")

        # Selection
        fitnesses.sort(key=lambda x: x[0])
        population = [cand for _, cand in fitnesses[:pop_size // 2]]

        # Crossover and mutation
        new_population = []
        while len(new_population) < pop_size:
            parents = random.sample(population, 2)
            c1 = crossover(parents[0], parents[1])
            c1 = mutate(c1, pass_pool)
            # c2 = mutate(c2, pass_pool)
            new_population.extend([c1])

        population = new_population

    # Final best
    best = min(fitnesses, key=lambda x: x[0])
    print(f"Best candidate: {best[1]} with fitness {best[0]}")


## Example usage

In [10]:
PASS_POOL = [
    "adce",
    "always-inline",
    "argpromotion",
    "block-placement",
    "break-crit-edges",
    "codegenprepare",
    "constmerge",
    "dce",
    "deadargelim",
    "dse",
    "function-attrs",
    "globaldce",
    "globalopt",
    "gvn",
    "indvars",
    "inline",
    "instcombine",
    "aggressive-instcombine",
    "internalize",
    "ipsccp",
    "normalize",
    "jump-threading",
    "lcssa",
    "licm",
    "loop-deletion",
    "loop-extract",
    "loop-reduce",
    "loop-rotate",
    "loop-simplify",
    "loop-unroll",
    "loop-unroll-and-jam",
    "lower-global-dtors",
    "lower-atomic",
    "lower-invoke",
    "lower-switch",
    "mem2reg",
    "memcpyopt",
    "mergefunc",
    "mergereturn",
    "partial-inliner",
    "reassociate",
    "rel-lookup-table-converter",
    "reg2mem",
    "sroa",
    "sccp",
    "simplifycfg",
    "sink",
    "simple-loop-unswitch",
    "strip",
    "strip-dead-debug-info",
    "strip-dead-prototypes",
    "strip-nondebug",
    "tailcallelim"
]

In [ ]:
sources = ["src/aha-mont64/mont64.c"]
output_bc = "scratch/test.bc"

compile_to_bitcode(sources,output_bc)



run_ga(output_bc, PASS_POOL, generations=5, pop_size=10, seq_length=5)

Compiling to LLVM bitcode: scratch/test.bc
Running: clang -target riscv64 -march=rv64ima -mabi=lp64 --sysroot=/opt/riscv_ima/riscv64-unknown-elf -emit-llvm -c -O0 -I./support/ src/aha-mont64/mont64.c -o scratch/test.bc


In [ ]:
clang -target riscv64 \
  --sysroot=/opt/riscv_ima/riscv64-unknown-elf \
  -march=rv64ima -mabi=lp64 \
  -emit-llvm -c -O0 \
  -I./support/ \
  src/aha-mont64/mont64.c \
  -o scratch/mont64.bc


clang -target riscv64 \
  --sysroot=/opt/riscv_ima/riscv64-unknown-elf \
  -march=rv64ima -mabi=lp64 -msoft-float \
  -emit-llvm -c -O0 \
  -I./support/ \
  support/beebsc.c \
  -o scratch/beebsc.bc
